In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import keras
from keras.preprocessing.image import ImageDataGenerator, img_to_array , load_img
from keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions

In [ ]:
train_datagen = ImageDataGenerator(zoom_range = 0.5, shear_range = 0.3, preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
t_path = "/content/drive/MyDrive/data/train/"
v_path = "/content/drive/MyDrive/data/val/"

train = train_datagen.flow_from_directory(directory = t_path , target_size = (256, 256), batch_size = 32)

val = train_datagen.flow_from_directory(directory = v_path , target_size = (256, 256), batch_size = 32)

In [ ]:
t_img, label = train.next();

In [ ]:

def plotImage(img_arr, label):
  for i, l in zip(img_arr, label):
    plt.figure(figsize=(5,5))
    plt.imshow(i/255)
    plt.show()

In [ ]:
plotImage(t_img[:3], label[:3])

# building the model

In [ ]:
from keras.layers import Dense, Flatten
from keras.models import Model
import keras

In [ ]:
base_model = VGG19(input_shape=(256,256,3), include_top=False)

In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(base_model.output)

x = Dense(units = 4, activation="softmax" )(x)

#creating model
model = Model (base_model.input, x)

model.summary()

In [ ]:
model.compile(optimizer="adam", loss=keras.losses.categorical_crossentropy, metrics = ['accuracy'] )

# Early stopping and Model check point

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# ealry stopping

es = EarlyStopping(monitor = "val_accuracy", min_delta = 0.01, patience = 1, verbose = 1)

# model check point
mc = ModelCheckpoint(filepath="best_model.h5", save_best_only=True, 
                     monitor = "val_accuracy", min_delta = 0.01, patience = 1, verbose = 1 )


cb = [es, mc]


In [ ]:
his = model.fit_generator(train, 
                          steps_per_epoch=16, 
                          epochs=50, 
                          verbose=1, 
                          callbacks=cb, 
                          validation_data=val, 
                          validation_steps=16)

In [ ]:
h = his.history
h.keys()

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c='red')
plt.title("acc vs v-acc")
plt.show()

In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c='red')
plt.title("loss vs v-loss")
plt.show()

In [ ]:
# load best models

from keras.models import load_model

model = load_model('/content/best_model.h5')

In [ ]:
acc = model.evaluate_generator(val)[1]

print(f"[INFO]: The accuracy of the model is: {acc * 100}%")

In [ ]:
ref = dict(zip(list(train.class_indices.values()) ,list(train.class_indices.keys())))

In [ ]:
def predict(path):
  img = load_img(path, target_size=(256,256))
  i = img_to_array(img)
  im = preprocess_input(i)
  img = np.expand_dims(im, axis = 0)
  # print(img.shape)
  pred = np.argmax(model.predict(img))
  
  print(f"The Image belongs to: {ref[pred]}")


In [ ]:
path = "/content/drive/MyDrive/data/test/fresh cotton plant/dsd (632).jpg"
predict(path)